# Optimizing Dry Bean Sorting into Classes using Classification with K-nearest neighbors

## Introduction:

Checklist (change)
1. provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report (DONE)
2. clearly state the question you tried to answer with your project (DONE)
3. identify and describe the dataset that was used to answer the question (DONE)



Dry beans, scientifically known as Phaseolus vulgaris L., are a crucially important and widely produced pulse globally (Beebe et al., 2013). In Turkish agriculture, dry beans play a significant role as a dinner table staple in many households. Turkey has several varieties of dry beans, including Barbunya, Battal, Bombay, Calı, Dermason, Horoz, Tombul, Selanik, and Seker. However, the market value of dry beans depends on the specific type of bean, and some beans are sold for higher prices than others.

The cultivation of the seven different beans together makes sorting them a crucial task (Koklu & Ozkan, 2020). Rather than selling them as a mixed batch, farmers can make more profit by sorting them by bean type. Manual classification and sorting of bean seeds can be tedious and time-consuming, especially when dealing with high production volumes. Manual sorting is costly and can vary in accuracy by individual, which has led to growing interest in automatic sorting systems. Computer algorithms can analyze images of seeds and classify them based on features like size and shape, making sorting faster, more accurate, and less labor-intensive than manual sorting.

In this project, we are interested in seed classification within the context of our dataset, which identifies the seven different types of dry beans cultivated in Turkey based on their physical features according to the Turkish Standards Institution. This research uses images of seven varieties of dry beans generated by a computer vision system, examining 16 features of the grains (Koklu & Ozkan, 2020). Our aim in this project is to create a k-NN classifier that can classify dry bean seeds.


![](https://ars.els-cdn.com/content/image/1-s2.0-S0168169919311573-gr3.jpg)

source: https://www.sciencedirect.com/science/article/pii/S0168169919311573

**Predictive question**: Can we accurately predict bean type (Seker, Barbunya, Bombay, Cali, Dermosan, Horoz and Sira) using physical features of the dry beans?

### Variables of Interest:

| Variable Name  | Description                                 |
|--------|---------------------------------------------|
| Area (A)     | The area of a bean zone and the number of pixels within its boundaries.            |
|Perimeter (P)    | Bean circumference is defined as the length of its border.         |
| Major axis length (L)    | The distance between the ends of the longest line that can be drawn from a bean     |
|  Minor axis length (l)    | The longest line that can be drawn from the bean while standing perpendicular to the main axis |
| Aspect ratio (K)     | Defines the relationship between L and l           |
|Eccentricity (Ec)    | Eccentricity of the ellipse having the same moments as the region        |
| Convex area (C)    | Number of pixels in the smallest convex polygon that can contain the area of a bean seed      |
|  Equivalent diameter (Ed)   | The diameter of a circle having the same area as a bean seed area. |
| Extent (Ex)    |The ratio of the pixels in the bounding box to the bean area.            |
|Solidity (S)    | Also known as convexity. The ratio of the pixels in the convex shell to those found in beans          |
| Roundness (R)    |Calculated with the following formula: (4piA)/(P^2)   |
|  Compactness (CO)    | |
| ShapeFactor1 (SF1)    |          |
|ShapeFactor2 (SF2)    |           |
| ShapeFactor3 (SF3)   |       |
|  ShapeFactor4 (SF4)   | |





### Methods & Results:

Checklist
1. loads data from the original source on the web -- (DONE) 
2. wrangles and cleans the data from it's original (downloaded) format to the format necessary for the planned analysis -- (DONE) 
3. performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis -- (DONE)
4. creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis -- (DONE)
5. performs the data analysis
6. creates a visualization of the analysis 
7. note: all tables and figure should have a figure/table number and a legend

In [113]:
# Download libraries
library(tidyverse)
library(tidymodels)
library("dplyr")
library("tidyr")
library("ggplot2")
options(repr.matrix.max.rows = 6)
install.packages("visdat")
library(visdat)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [114]:
# set seed
set.seed(25)

In [115]:
# Load data 
url <- "https://github.com/NaziaNadir/dsci-100-2023-007-group-8/raw/main/Dry_Bean_Dataset.csv"
bean_data <- read_csv(url)
bean_data

Rows: 13611 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): Class
dbl (16): Area, Perimeter, MajorAxisLength, MinorAxisLength, AspectRation, E...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
28395,610.291,208.1781,173.8887,1.197191,0.5498122,28715,190.1411,0.7639225,0.9888560,0.9580271,0.9133578,0.007331506,0.003147289,0.8342224,0.9987239,SEKER
28734,638.018,200.5248,182.7344,1.097356,0.4117853,29172,191.2728,0.7839681,0.9849856,0.8870336,0.9538608,0.006978659,0.003563624,0.9098505,0.9984303,SEKER
29380,624.110,212.8261,175.9311,1.209713,0.5627273,29690,193.4109,0.7781132,0.9895588,0.9478495,0.9087742,0.007243912,0.003047733,0.8258706,0.9990661,SEKER
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
42139,759.321,281.5399,191.1880,1.472582,0.7340648,42569,231.6313,0.7299324,0.9898988,0.9184241,0.8227297,0.006681220,0.001888271,0.6768842,0.9967673,DERMASON
42147,763.779,283.3826,190.2757,1.489326,0.7410548,42667,231.6532,0.7053891,0.9878126,0.9079065,0.8174575,0.006723673,0.001852025,0.6682367,0.9952224,DERMASON
42159,772.237,295.1427,182.2047,1.619841,0.7866930,42600,231.6862,0.7889625,0.9896479,0.8883804,0.7849972,0.007000705,0.001639812,0.6162206,0.9981796,DERMASON


In [116]:
#Wrangle/clean data 

clean_bean_data<-bean_data|>
mutate(Class=as_factor(Class))

clean_bean_data<-na.omit(clean_bean_data)

clean_bean_data

Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
28395,610.291,208.1781,173.8887,1.197191,0.5498122,28715,190.1411,0.7639225,0.9888560,0.9580271,0.9133578,0.007331506,0.003147289,0.8342224,0.9987239,SEKER
28734,638.018,200.5248,182.7344,1.097356,0.4117853,29172,191.2728,0.7839681,0.9849856,0.8870336,0.9538608,0.006978659,0.003563624,0.9098505,0.9984303,SEKER
29380,624.110,212.8261,175.9311,1.209713,0.5627273,29690,193.4109,0.7781132,0.9895588,0.9478495,0.9087742,0.007243912,0.003047733,0.8258706,0.9990661,SEKER
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
42139,759.321,281.5399,191.1880,1.472582,0.7340648,42569,231.6313,0.7299324,0.9898988,0.9184241,0.8227297,0.006681220,0.001888271,0.6768842,0.9967673,DERMASON
42147,763.779,283.3826,190.2757,1.489326,0.7410548,42667,231.6532,0.7053891,0.9878126,0.9079065,0.8174575,0.006723673,0.001852025,0.6682367,0.9952224,DERMASON
42159,772.237,295.1427,182.2047,1.619841,0.7866930,42600,231.6862,0.7889625,0.9896479,0.8883804,0.7849972,0.007000705,0.001639812,0.6162206,0.9981796,DERMASON


In [117]:
# Data set exploration summary

In [118]:
# Name of each category in Class column
clean_bean_data |>
  pull(Class) |>
  levels()

[1] "SEKER"    "BARBUNYA" "BOMBAY"   "CALI"     "HOROZ"    "SIRA"     "DERMASON"

In [119]:
# Number and percentage of each category.
 
num_obs <- nrow(clean_bean_data)

bean_number<-clean_bean_data|>
  group_by(Class) |>
  summarize(
    count = n(),
    percentage = n() / num_obs * 100)
print(bean_number)



# A tibble: 7 × 3
  Class    count percentage
  <fct>    <int>      <dbl>
1 SEKER     2027      14.9 
2 BARBUNYA  1322       9.71
3 BOMBAY     522       3.84
4 CALI      1630      12.0 
5 HOROZ     1928      14.2 
6 SIRA      2636      19.4 
7 DERMASON  3546      26.1 


Results from the table:
The Class count is not too different so we do not need to resample our data 

In [120]:
#Create the train/ test split
bean_split <- initial_split(final_bean_data, prop = 0.50, strata = Class)
bean_train <- training(bean_split)
bean_test <- testing(bean_split)

In [121]:
glimpse(bean_train)

Rows: 1,699
Columns: 10
$ Area          <dbl> 50575, 59100, 62839, 64530, 66062, 70743, 71973, 74864, …
$ AspectRation  <dbl> 1.529822, 1.635922, 1.620561, 1.676689, 1.515300, 1.4126…
$ Compactness   <dbl> 0.8073124, 0.7794548, 0.7843610, 0.7713281, 0.8108868, 0…
$ ConvexArea    <dbl> 51389, 60703, 63383, 65642, 67275, 71718, 72745, 75820, …
$ Eccentricity  <dbl> 0.7567789, 0.7914174, 0.7869081, 0.8026774, 0.7513226, 0…
$ EquivDiameter <dbl> 253.7599, 274.3145, 282.8588, 286.6394, 290.0220, 300.12…
$ Extent        <dbl> 0.7121233, 0.7787074, 0.7987061, 0.6979234, 0.7086297, 0…
$ Perimeter     <dbl> 858.520, 1057.719, 952.704, 1004.942, 1010.706, 1034.348…
$ roundness     <dbl> 0.8622733, 0.6638298, 0.8700078, 0.8029519, 0.8126656, 0…
$ Class         <fct> BARBUNYA, BARBUNYA, BARBUNYA, BARBUNYA, BARBUNYA, BARBUN…


In [122]:
glimpse(bean_test)

Rows: 1,702
Columns: 10
$ Area          <dbl> 53874, 62005, 62353, 68460, 68942, 74151, 159806, 173366…
$ AspectRation  <dbl> 1.548807, 1.549585, 1.423070, 1.577395, 1.544634, 1.5377…
$ Compactness   <dbl> 0.8027612, 0.8016634, 0.8375467, 0.7913757, 0.8038210, 0…
$ ConvexArea    <dbl> 54878, 62953, 63337, 69842, 70514, 75148, 164335, 175139…
$ Eccentricity  <dbl> 0.7636265, 0.7639003, 0.7114800, 0.7733687, 0.7621484, 0…
$ EquivDiameter <dbl> 261.9055, 280.9755, 281.7629, 295.2389, 296.2764, 307.26…
$ Extent        <dbl> 0.7365874, 0.7987144, 0.7674215, 0.7036694, 0.7130210, 0…
$ Perimeter     <dbl> 913.878, 960.799, 974.133, 1022.328, 1072.869, 1071.243,…
$ roundness     <dbl> 0.8106113, 0.8440564, 0.8257160, 0.8231259, 0.7526626, 0…
$ Class         <fct> BARBUNYA, BARBUNYA, BARBUNYA, BARBUNYA, BARBUNYA, BARBUN…



We can observe from glimpse in the code above that the training set contains 6804 observations, while the test set contains 6807 observations. This corresponds to a train / test split of 50% / 50%, as desired.

# Summary Table

In [125]:
# use summary function to get basic statistics for each variable
summary_df <- summary(bean_train)

# Creat table using the data.frame function

table_df <- data.frame(n= summary_df[,"Length"],
                       mean=summary_df[,"Mean"],
                       sd=summary_df[,"Std.Dev"],
                       median=summary_df[,"Median"],
                       min=summary_df[,"Min."],
                       max=summary_df[,"Max."],
                       range=summary_df[,"Range"],
                       se=summary_df[,"Std.Error"])
row.names(table_df) <- c("Area", "Perimeter", "MajorAxisLength", "MinorAxisLength", "Eccentricity", "ConvexArea", "Extent", "Roundness")



ERROR: Error in `[.default`(summary_df, , "Length"): subscript out of bounds


In [ ]:
# Heatmap for correlation of variables
bean_train |> 
 select(where(is.numeric)) |>
  vis_cor()

In [ ]:
# Predictor selection
options(repr.plot.height = 9, repr.plot.width = 15)
bean_train |>
    pivot_longer(cols = -Class, names_to = 'category', values_to = 'value')  |> 
    ggplot(aes(value, fill = Class)) +
        geom_density(alpha = 0.4, colour = 'white') +
        facet_wrap(facets = vars(category), scales = 'free') +
        theme_minimal() +
        labs(x="Distribution of Bean Type across Variables", y="Count", fill="Bean Type")+
        ggtitle("Figure 1: Using Density Plots To Perform Variable Section")+
        theme(text = element_text(size = 20)) +
 theme(axis.text.x = element_text(angle=40, hjust =1)) +
  scale_x_log10(labels = label_comma())

To choose our predictors:

First, we removed variables we don't understand. These include shape factors 1, 2, 3, and 4.

Next, we removed variables that convey similar things making them redundant. When it comes to using them as predictors, this will cause our model to place a greater emphasis on those predictors since they're counted twice.

The aspect ratio is the relationship between the major and minor axis of an object. So, we removed the major axis and minor axis.
Area is similar to perimeter. So, we removed the area.
Lastly, we utilized histograms to visualize the distribution of different variables. If the distributions between the graphs greatly overlap then it might not be a good predictor because if an unknown observation was used, it'd be within a cluster of many different bean types. If one distribution was skewed to the right and one distribution was skewed to the left, i.e. very little overlap that could be an indication of a good predictor. Referring to figure 1, distribution of the solidity graph shows notable overlap between every class of bean; thus, this variable was removed.

Therefore, we are left with and will be using the following variables to be used in our classifier: Area, AspectRation, Compactness, ConvexArea, Eccentricity, EquivDiameter, Extent, Perimeter, and Roundness.

In [ ]:
#Update data set
final_bean_data<- bean_train|>
select("Area", "AspectRation", "Compactness", "ConvexArea", "Eccentricity",
       "EquivDiameter", "Extent", "Perimeter", "roundness", "Class")
final_bean_data

In [ ]:
bean_vfold <- vfold_cv(bean_train, v = 5, strata = Class)

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")
knn_spec

In [ ]:
bean_recipe <- recipe(Class ~ ., data = bean_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())
bean_recipe

In [ ]:
k_vals <- tibble(neighbors = seq(from = 1, to = 20, by = 1))
k_vals

In [ ]:
knn_results <- workflow() |>
  add_recipe(bean_recipe) |>
  add_model(knn_spec) |>
  tune_grid(resamples = bean_vfold, grid = k_vals) |>
  collect_metrics() 

accuracies <- knn_results |>
  filter(.metric == "accuracy")

accuracies
knn_results 

In [ ]:
accuracies_max <- accuracies |>
arrange(mean)|>
tail(1)
accuracies_max

In [ ]:
accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate") + 
  theme(text = element_text(size = 12))+
ggtitle("Figure 2: Selecting the Best K")

accuracy_vs_k

In [ ]:
#NOTE: I added this version of the cross validation plot to see if it may be visually better
#Adjust the k value if we change best k = 10 to something else
options(repr.plot.width = 12, repr.plot.height = 7)

ky <- accuracies |>
    filter(neighbors == 18) |>
    select(mean) |>
    pull()

cross_val_plot <- accuracies |> 
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    geom_vline(xintercept = 18, colour = "red") +
    geom_point(aes(x = 18, y = ky), size = 4, colour = "blue") +
    labs(x = "\nNeighbours",
         y = "Accuracy Estimate\n",
         title = "Estimated Accuracies of k Neighbours") +
    scale_x_continuous(limits = c(1, 25),
                       breaks = seq(1, 25),
                       minor_breaks = seq(1, 25, 1)) +
    theme(text = element_text(size = 20),
          plot.caption = element_text(size = 20, hjust = 0))

cross_val_plot

Based on the findings presented in Figure 2, it can be concluded that a value of k=10 provides the highest estimated accuracy. Moreover, the graph indicates that the estimated accuracy remains relatively stable even if we slightly increase or decrease k around the value of 10. Furthermore, selecting k=10 does not result in a computationally expensive training process. Therefore, taking all these factors into consideration, we would choose k=10 as the optimal value for the classifier.

In [ ]:
# Make a new model specification for the best parameter value 

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 10) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_fit <- workflow() |>
  add_recipe(bean_recipe) |>
  add_model(knn_spec) |>
  fit(data = bean_train)
knn_fit

In [ ]:
# Evaluate the estimated accuracy of the classifier 

In [ ]:
bean_predictions <- predict(knn_fit, bean_test) |>
  bind_cols(bean_test)|>
mutate(Class = as_factor(Class))

bean_predictions

In [ ]:
levels(bean_predictions$.pred_class) <- levels(bean_predictions$Class)
bean_predictions |>
metrics(truth = Class, estimate =.pred_class)|>
filter(.metric == "accuracy")

# Summary Table

In [ ]:
confusion <- bean_predictions |>
             conf_mat(truth = Class, estimate = .pred_class)
confusion



In [ ]:
# Heat map based on the confusion matrix
z =c(950,  6,  0,    2,     0,   20 ,      46,
      4,   556,      0,   26,     4,    4,  0,
      0,    0,    256,    0,     0,    0,    0,
      0,    69,      0,  780,    19,    2,    0,
      0,     3,      0,    7,   900,   10,    2,
      39,    13,      0,    7,    36, 1181,   121,
      17,     0,      0,   0,     5,  115,    1607)
z = matrix(z, ncol=7)
colnames(z) = c("SEKER", "BARBUNYA", "BOMBAY", "CALI", "HOROZ", "SIRA", "DERMASON")
rownames(z) = c("SEKER", "BARBUNYA", "BOMBAY", "CALI", "HOROZ", "SIRA", "DERMASON")


my_colors <- colorRampPalette((c("green", "red")))
heatmap(z, col=my_colors(10000000),Rowv=NA, Colv=NA)


### Discussion:

Checklist vv
1. summarize what you found
2. discuss whether this is what you expected to find?
3. discuss what impact could such findings have?

The selection of the optimal seed species is a crucial issue for bean farmers and the market alike. With a wide range of genotypes cultivated across the globe, it is imperative to differentiate the finest seed variety from the mixed dry bean population. If this is not done, the market value of these heterogeneous bean species may witness a significant decline, as pointed out by Varankaya and Ceyhan (2012). Findings from this study have significant implications for the agricultural industry, as development of automated image classification technology can improve efficiency and accuracy of dry bean sorting, reducing labor costs. Success could lead to research on applications of image classification models to other agricultural products (e.g., crops and fruits), or examining socio-economic impacts of this technology via agriculture, specifically potential for job displacement.

4. discuss what future questions could this lead to?

### References

Checklist
1. At least 2 citations of literature relevant to the project (format is your choice, just be consistent across the references).
2. Make sure to cite the source of your data as well.

Varankaya, S., & Ceyhan, E. (2012). Problems Encountered in Bean Farming in the Central Anatolia Region and Solution Suggestions. Selçuk Tarım Bilim. Journal. 26, 15–26.